### Code used for reading and converting PCraster files to array


#### This code provide an automatic read of PCraster files into array, and then organize the data in a time series to be used for further analysis.

#### The code will read a total of Z .map files, and will reshape each file from a (X, Y) to a (1, X * Y), and will assing each file matrix to a different line. Therefore we will have a time-series matrix of (Z, X * Y) shape. 

#### Since the full raster matrix is too big, a clone map mask is used to convert just the grids that are from rivers and streams in the map, reducing the number of columns. 

Developed by: Thiago Victor Medeiros do Nascimento

In [1]:
from pcraster import *
import numpy as np
from osgeo import gdal, gdalconst
from osgeo import gdal_array
from osgeo import osr
import matplotlib.pylab as plt
import subprocess
import glob,os
import time

In [2]:
path =r'D:\pythonDATA\AquadaptTemez\rastermapaccum'
filenames = glob.glob(path + "/*.map")
len(filenames)

1032

In [3]:
# time series matrix 
mapfile = filenames[0]

RasterLayer = gdal.Open(mapfile)

ncols = RasterLayer.RasterXSize
nrows = RasterLayer.RasterYSize

# about 50 arrays
numtotal = nrows*ncols
numtotal

48694800

We cannot process all the data efficiently, and besides we are interested mainly on data in the river cells, therefore we can use a clone map with a filter. This map has 1 as river cells and 0 as non-river cells. Therefore, the processing will be optimizes solely for the river cells. 

In [4]:
pathfilter =r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicatorsmap\rivernetworkabove5000mmclipped.map'
mapfilter = readmap(pathfilter)
Rastermapfilter = gdal.Open(pathfilter)
mapfilterarray = pcr_as_numpy(mapfilter)
mapfilterarray
mapfilterarray[mapfilterarray < 1 ] = np.nan

newnumtotal = np.count_nonzero(~np.isnan(mapfilterarray))

# We need additionally to reshape our filter:
mapfilterarrayres = np.reshape(mapfilterarray, (1, numtotal))
mapfilterarrayres

array([[nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [5]:
print("We initially had a total of:", numtotal, "cells, however only", newnumtotal, "refers indeed for river cells")

We initially had a total of: 48694800 cells, however only 2301110 refers indeed for river cells


In [6]:
# We create an array with the total number of non NaNs as columns and the total months as rows:
runofftsarray = np.zeros((len(filenames),newnumtotal),dtype=np.float32)

This loop will read each month (.map file) transform it in a array, and exctract solely the river cells data for our runofftsarray:

In [ ]:
start = time.time()
for mapfile in filenames:

    
    namewithmap = os.path.basename(mapfile)
    namemap = namewithmap.replace("accum.map", "")
    namemap = namemap.replace("T", "")
    
    # The files are not organized in order when they are read, therefore we will make a way to write each line in the correct line of the geral
    namemapint = int(namemap) - 1 
    
     
    mapreadarray = pcr_as_numpy(readmap(mapfile))
    mapreadarrayres = np.reshape(mapreadarray, (1, numtotal))
    
    
    
    runofftsarray[namemapint,:] = mapreadarrayres[~np.isnan(mapfilterarrayres)]
end = time.time()
print(end - start)

Finally, the time-series matrix is saved as a CSV file to be further analysed

In [9]:
np.savetxt(r'D:\pythonDATA\AquadaptTemez\runofftsarray.csv', runofftsarray, delimiter=',')